## Setup environment

In [1]:
import os
import json
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

In [2]:
config_file = 'config.json'

with open(config_file, 'r') as f:
    config = json.load(f)

ga_data_dir = config["ga_data_dir"]
fdg_data_dir = config["fdg_data_dir"]
log_dir = config["log_dir"]
ga_output_dir = config["ga_output_dir"]
artifact_dir = config["artifacts"]
artifact_output_dir = config ["artifact_output"]
fdg_output_dir = config['fdg_output_dir']


## Set dataset path

------
# FDG

In [28]:

from data_preparation import ExternalRadioSetHandling

data_handler = ExternalRadioSetHandling(data_dir=fdg_data_dir, test_ratio=0.2)
train, valid, test_files = data_handler.get_split_data()

print(f"Test data length: {len(test_files)}")

# To access 'image' keys, iterate through the list
image_files = [file_info['image'] for file_info in test_files]
target_files = [file_info['target'] for file_info in test_files]
print(len(image_files))

Test data length: 21
21


In [32]:
from data_preparation import ExternalRadioSetHandling

data_handler = ExternalRadioSetHandling(data_dir=fdg_data_dir, test_ratio=0.2)
test_files, _, _ = data_handler.get_split_data()

print(f"Test data length: {len(test_files)}")

# To access 'image' keys, iterate through the list
image_files = [file_info['image'] for file_info in test_files]
target_files = [file_info['target'] for file_info in test_files]
print(len(image_files))
test_name = [(os.path.splitext(os.path.splitext(os.path.basename(file_info['image']))[0])[0], 
              os.path.splitext(os.path.splitext(os.path.basename(file_info['target']))[0])[0]) 
             for file_info in test_files]
test_name

Test data length: 62
62


[('006009_dataset_07_A', '006009_dataset_07_A'),
 ('005081_dataset_06_A_nac_6', '005081_dataset_06_A_nac_6'),
 ('006056_dataset_07_A', '006056_dataset_07_A'),
 ('005035_dataset_06_A', '005035_dataset_06_A'),
 ('006036_dataset_07_A', '006036_dataset_07_A'),
 ('006038_dataset_07_A', '006038_dataset_07_A'),
 ('006011_dataset_07_A', '006011_dataset_07_A'),
 ('005085_dataset_06_A_nac_6', '005085_dataset_06_A_nac_6'),
 ('005019_dataset_06_A', '005019_dataset_06_A'),
 ('005001_dataset_06_A', '005001_dataset_06_A'),
 ('005065_dataset_06_A', '005065_dataset_06_A'),
 ('006025_dataset_07_A', '006025_dataset_07_A'),
 ('005096_dataset_06_A_nac_6', '005096_dataset_06_A_nac_6'),
 ('005049_dataset_06_A_nac_3', '005049_dataset_06_A_nac_3'),
 ('005091_dataset_06_A_nac_6', '005091_dataset_06_A_nac_6'),
 ('005008_dataset_06_A', '005008_dataset_06_A'),
 ('005041_dataset_06_A_nac_3', '005041_dataset_06_A_nac_3'),
 ('005021_dataset_06_A', '005021_dataset_06_A'),
 ('006048_dataset_07_A', '006048_dataset_07_A'

--------
# Ga-Artifact

In [25]:
from data_preparation import ExternalRadioSetHandling

data_handler = ExternalRadioSetHandling(data_dir=artifact_dir, test_ratio=1)
_, _, test_files = data_handler.get_split_data()

print(f"Test data length: {len(test_files)}")

# To access 'image' keys, iterate through the list
image_files = [file_info['image'] for file_info in test_files]
target_files = [file_info['target'] for file_info in test_files]
print(len(image_files))

Test data length: 13
13


----------

In [29]:
test_name = [(os.path.splitext(os.path.splitext(os.path.basename(file_info['image']))[0])[0], 
              os.path.splitext(os.path.splitext(os.path.basename(file_info['target']))[0])[0]) 
             for file_info in test_files]
test_name

[('006006_dataset_07_A', '006006_dataset_07_A'),
 ('006010_dataset_07_A', '006010_dataset_07_A'),
 ('006051_dataset_07_A', '006051_dataset_07_A'),
 ('005073_dataset_06_A', '005073_dataset_06_A'),
 ('005060_dataset_06_A_nac_6', '005060_dataset_06_A_nac_6'),
 ('005002_dataset_06_A_nac_6', '005002_dataset_06_A_nac_6'),
 ('005106_dataset_06_A', '005106_dataset_06_A'),
 ('005039_dataset_06_A', '005039_dataset_06_A'),
 ('005005_dataset_06_A', '005005_dataset_06_A'),
 ('005043_dataset_06_A_nac_3', '005043_dataset_06_A_nac_3'),
 ('006044_dataset_07_A', '006044_dataset_07_A'),
 ('006046_dataset_07_A', '006046_dataset_07_A'),
 ('006024_dataset_07_A', '006024_dataset_07_A'),
 ('006041_dataset_07_A', '006041_dataset_07_A'),
 ('006037_dataset_07_A', '006037_dataset_07_A'),
 ('005037_dataset_06_A', '005037_dataset_06_A'),
 ('006007_dataset_07_A', '006007_dataset_07_A'),
 ('006021_dataset_07_A', '006021_dataset_07_A'),
 ('005026_dataset_06_A', '005026_dataset_06_A'),
 ('006053_dataset_07_A', '006053_

---------
# Create Corrected Images

In [ ]:
from data_preparation import LoaderFactory

loader_factory = LoaderFactory(
    train_files=_,
    val_files=_,
    test_files=test_files,
    patch_size = [168, 168, 16],
    spacing = [4.07, 4.07, 3.00],
    spatial_size = (168, 168, 600) 
    )


test_loader = loader_factory.get_loader('test', batch_size=1, num_workers=2, shuffle=False)


In [ ]:
log_filename = 'log_4_17_19_29.txt'  # Ga, new structure. editesh kardam payini ro ezaf kardam behesh.

bestmodel_filename =  'model_4_24_23_17.pth'
best_metric = 0.0527
best_epoch = 434

In [ ]:
import os
import torch
from model_maker import get_network, add_activation_before_output
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = get_network(patch_size = [168, 168, 16], spacing = [4.07, 4.07, 3.00])

model = model.to(device)

add_activation_before_output(model, nn.ReLU(inplace=True))
model

In [ ]:
import torch

# Assuming root_dir is the directory where your model files are stored
model_path = os.path.join(log_dir, bestmodel_filename)
if os.path.exists(model_path):
    print(f"Model file {bestmodel_filename} is loading.")
    model.load_state_dict(torch.load(model_path))
    model.eval()
else:
    print(f"Model file {bestmodel_filename} not found.")

In [ ]:

from monai.inferers import sliding_window_inference
import matplotlib.pyplot as plt
import numpy as np
from monai.transforms import Compose, Invertd, SaveImaged
from monai.data import decollate_batch


roi_size = (168, 168, 16)
sw_batch_size = 64


post_transforms = Compose(
    [
        Invertd(
            keys="pred",
            transform=loader_factory.get_test_transforms(),
            orig_keys="image",
            meta_keys="pred_meta_dict",
            orig_meta_keys="image_meta_dict",
            meta_key_postfix="meta_dict",
            nearest_interp=False,
            to_tensor=True,
        ),
        # ClampNegative(keys=["pred"]),
        SaveImaged(keys="pred", meta_keys="pred_meta_dict", output_dir=artifact_output_dir, output_postfix="dl4_23", resample=False), 
    ]
)
# comb_3_27_onfdg_v2 fdg_output_dir
# dl4_23 ga_output_dir
# comb_4_5_onfdg fdg_output_dir

with torch.no_grad():
    for test_data in test_loader:
        test_inputs = test_data["image"].to(device)
        test_data["pred"] = sliding_window_inference(test_inputs, roi_size, sw_batch_size, model, overlap=0.90)
        post_processed = [post_transforms(i) for i in decollate_batch(test_data)]



In [ ]:
from utils import find_dl_image_path, normalize_data
from vis import display_patient_coronal, display_patient_transverse


hint = "dl_final_3_29"
hint = 'gamodel_3_18_onfdg'
hint = 'dl4_23'
i = 1
for file_info, name_pair in zip(test_files, test_name):
    image_path = file_info['image']
    target_path = file_info['target']
    patient_folder_name_image, _ = name_pair

    dl_image_path = find_dl_image_path(artifact_output_dir, patient_folder_name_image, hint)
    if dl_image_path is None:
        print(f"No DL image found for {patient_folder_name_image}")
        continue

    # image = normalize_data(nib.load(image_path).get_fdata())
    # target = normalize_data(nib.load(target_path).get_fdata())
    # dl_image = normalize_data(nib.load(dl_image_path).get_fdata())

    image = nib.load(image_path).get_fdata()
    target = nib.load(target_path).get_fdata()
    dl_image = nib.load(dl_image_path).get_fdata()
    difference_image = target - dl_image

    difference_image = np.clip(difference_image, -1, 1)

    n = 117
    n = 101
    display_patient_coronal(patient_folder_name_image, image, target, dl_image, difference_image, n, cmp ="jet")
    # i +=1
    # if i == 4:
    #     break


    # n = 198

    # display_patient_transverse(patient_folder_name_image, image, target, dl_image, difference_image, n)
    # # i +=1
    # # if i == 4:
    # #     break



In [ ]:
# import nibabel as nib
# import matplotlib.pyplot as plt
# import numpy as np
# import os
# import glob
# from matplotlib.colors import LinearSegmentedColormap
# import numpy as np
# import matplotlib.pyplot as plt

# def apply_window(image, window_level, window_width):
#     """
#     Apply window level and width to an image.

#     Parameters:
#     - image: The input image as a NumPy array.
#     - window_level (WL): The center of the range of interest.
#     - window_width (WW): The width of the range of interest.

#     Returns:
#     - The windowed image.
#     """
#     lower_bound = window_level - window_width / 2
#     upper_bound = window_level + window_width / 2
#     windowed_image = np.clip(image, lower_bound, upper_bound)
#     windowed_image = (windowed_image - lower_bound) / window_width
#     return windowed_image

# window_level = 0  # Example values
# window_width = 1  # Example values

# hint = "dl3_18"

# # Function to find the DL image path for a given patient folder name
# def find_dl_image_path(patient_folder_name):
#     # Construct a glob pattern to search for DL images with the matching patient folder name
#     search_pattern = os.path.join(artifact_output, "**", f"{patient_folder_name}*{hint}.nii.gz")
#     found_paths = glob.glob(search_pattern, recursive=True)
#     if found_paths:
#         return found_paths[0]  # Return the first match
#     else:
#         return None  # No match found

# def normalize_data(data):
#     return (data - np.min(data)) / (np.max(data) - np.min(data))
# cmap = LinearSegmentedColormap.from_list(
#     'bwr', 
#     # ['purple', 'purple','darkviolet','white','white', 'lime', 'limegreen','limegreen']
#     ['purple','white','limegreen']
#     # ['purple', 'purple','purple','white','white', 'limegreen', 'limegreen','limegreen']
#     #['red', 'red','orangered','white','white', 'royalblue', 'blue','blue']
#     #['red', 'red','red','white','white', 'blue', 'blue','blue']
#     # ['red', 'red','red','tomato','white','white' , 'dodgerblue', 'blue', 'blue', 'blue']
# )
# # Loop over each file pair, find the matching DL image, and plot them
# for file_info, name_pair in zip(test_files, test_name):
#     image_path = file_info['image']
#     target_path = file_info['target']
#     patient_folder_name_image, _ = name_pair

#     dl_image_path = find_dl_image_path(patient_folder_name_image)
#     if dl_image_path is None:
#         print(f"No DL image found for {patient_folder_name_image}")
#         continue

#     # Load and normalize images
#     image = normalize_data(nib.load(image_path).get_fdata())
#     target = normalize_data(nib.load(target_path).get_fdata())
#     dl_image = normalize_data(nib.load(dl_image_path).get_fdata())
#     windowed_image = apply_window(image, window_level, window_width)
#     # Calculate the difference image and normalize to range -1 to 1
#     difference_image = target - dl_image
#     # difference_image = np.clip(difference_image, -1, 1)  # Ensure the difference is within -1 to 1

#     # difference_image = 2 * (difference_image - np.min(difference_image)) / (np.max(difference_image) - np.min(difference_image)) - 1

#     # image = nib.load(image_path).get_fdata()
#     # target = nib.load(target_path).get_fdata()
#     # dl_image = nib.load(dl_image_path).get_fdata()
#     # difference_image = target - dl_image
#     # # difference_image = np.clip(difference_image, -1, 1)  # Ensure the difference is within -1 to 1
#     n = 94

#     fig, axs = plt.subplots(1, 4, figsize=(20, 6))

#     plt.subplot(1, 4, 1)
#     plt.title(f"Input: {patient_folder_name_image}")
#     input_slice = np.rot90(image[:, n, :])
#     plt.imshow(input_slice, cmap="gist_yarg", vmin=0, vmax=0.1)
#     plt.axis('off')

#     plt.subplot(1, 4, 2)
#     plt.title("Target")
#     input_slice = np.rot90(target[:, n, :])
#     plt.imshow(input_slice, cmap="gist_yarg", vmin=0, vmax=0.1)
#     plt.axis('off')

#     plt.subplot(1, 4, 3)
#     plt.title("DL Image")
#     input_slice = np.rot90(dl_image[:, n, :])
#     plt.imshow(input_slice, cmap="gist_yarg", vmin=0, vmax=0.1)
#     plt.axis('off')

#     ax = plt.subplot(1, 4, 4)
#     plt.subplot(1, 4, 4)
#     plt.title("Difference")
#     # Using a diverging colormap to highlight differences
#     difference_display = plt.imshow(np.rot90(difference_image[:, n, :]), cmap=cmap, vmin=-0.1, vmax=0.1) # vmin=-np.max(np.abs(difference_image)), vmax=np.max(np.abs(difference_image)))
#     plt.axis('off')
#     fig.colorbar(difference_display, ax=axs[3])

#     plt.tight_layout()
#     plt.show()
#     # break
